In [ ]:
!nvidia-smi

In [ ]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

In [ ]:
import os

DATASET_ROOT = "/root/yolo_dataset"

print("Dataset root exists:", os.path.exists(DATASET_ROOT))
print("Images:", os.listdir(os.path.join(DATASET_ROOT, "images")))
print("Labels:", os.listdir(os.path.join(DATASET_ROOT, "labels")))

In [ ]:
import yaml
import os

NUM_CLASSES = 1
CLASS_NAMES = ["license_plate"]

data_yaml_content = dict(
    path=DATASET_ROOT,
    train="images/train",
    val="images/val",
    nc=NUM_CLASSES,
    names=CLASS_NAMES
)

YAML_PATH = os.path.join(DATASET_ROOT, "data.yaml")

with open(YAML_PATH, "w") as f:
    yaml.dump(data_yaml_content, f, default_flow_style=False)

print("✅ data.yaml saved at:", YAML_PATH)

!cat {YAML_PATH}

In [ ]:
import os
import subprocess
from ultralytics import YOLO

class AutoUploadCallback:
    def on_train_epoch_end(self, trainer):
        weight_dir = trainer.save_dir / "weights"
        subprocess.run([
            "rclone", "copy",
            str(weight_dir),
            "drive:/YOLO_CheckPOINTS",
            "--update", "-P"
        ])
        print("✅ Checkpoint uploaded to Drive")

In [ ]:
from ultralytics import YOLO

MODEL_WEIGHTS = "yolo11n.pt"   # hoặc yolo11n.pt nếu bạn có
model = YOLO(MODEL_WEIGHTS)
model.add_callback("on_train_epoch_end", AutoUploadCallback().on_train_epoch_end)

BATCH_SIZE = 16
NUM_EPOCHS = 100
IMAGE_SIZE = 640

RUN_NAME = "license_plate_rtx3060"
SAVE_DIR = "/root/yolo_runs"

results = model.train(
    data=YAML_PATH,
    epochs=NUM_EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    device=0,             # ✅ GPU RTX 3060
    workers=8,
    project=SAVE_DIR,
    name=RUN_NAME,
    resume=True
)

print("✅ Training finished")


In [ ]:
from ultralytics import YOLO
import os

RUN_NAME = "license_plate_rtx30602"
SAVE_DIR = "/root/yolo_runs"
CKPT_PATH = f"{SAVE_DIR}/{RUN_NAME}/weights/last.pt"

# Load checkpoint
model = YOLO(CKPT_PATH)
model.add_callback("on_train_epoch_end", AutoUploadCallback().on_train_epoch_end)

BATCH_SIZE = 16
IMAGE_SIZE = 640
TOTAL_EPOCHS = 20   # ✅ Mục tiêu cuối là 20 (bạn đang ở 15)

results = model.train(
    resume=True,             # ✅ bắt buộc
    epochs=TOTAL_EPOCHS,     # ✅ YOLO sẽ tự train từ 16 → 20
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    device=0,                # RTX 3060
    workers=8,
    project=SAVE_DIR,
    name=RUN_NAME,
    verbose=True,
    save=True,
    save_period=1
)

print("✅ Resume từ epoch 15 → 20 hoàn tất")